In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.visualize import get_pts_to_img

In [ ]:
img = pd.read_parquet("./data/raw/train_landmark_files/16069/100015657.parquet")

tmp = img[(img.frame == 107) | (img.frame == 106)]
data = np.array(tmp[["x", "y", "z"]]).reshape(2, 543, 3)

print(img.loc[img.x.isna()].query('frame == 103').type.value_counts())


In [ ]:
img.query('frame == 106 and type == "left_hand"')

In [ ]:
image = get_pts_to_img(data[1])
plt.imshow(image)


In [ ]:
from src.augmentations import aug_hflip, KeypointsIndex

In [ ]:
ndf = np.load("./data/generated/np_landmarks/100015657.npy")
ndf = ndf[5:7, :]

In [ ]:
x_pts_rh, y_pts_rh, z_pts_rh = ndf[:, KeypointsIndex.right_hand_x_index], ndf[:, KeypointsIndex.right_hand_y_index], ndf[:, KeypointsIndex.right_hand_z_index]
x_pts_lh, y_pts_lh, z_pts_lh = ndf[:, KeypointsIndex.left_hand_x_index], ndf[:,KeypointsIndex.right_hand_y_index], ndf[:, KeypointsIndex.left_hand_z_index]


In [ ]:
y_pts_lh

In [ ]:
1 - x_pts_lh

In [ ]:
ndf[:, KeypointsIndex.right_hand_x_index] = 1 - x_pts_lh
ndf[:, KeypointsIndex.left_hand_x_index] = 1 - x_pts_rh

In [ ]:
ndf[:, KeypointsIndex.left_hand_x_index]


In [ ]:
x_pts_rh.shape

In [ ]:
i = 0
for x in range(IndexLabels.right_hand[0], 3*IndexLabels.right_hand[1], 3):
    i += 1


In [ ]:
x_pts_rh.shape

In [ ]:
x_pts_rh, x_pts_lh = aug_hflip(data)

In [ ]:
ndf = np.load("./data/generated/np_landmarks/100015657.npy")


In [ ]:
data[:, IndexLabels.right_hand[0]: IndexLabels.right_hand[1]].shape

In [ ]:
1 - data[:, IndexLabels.left_hand[0]: IndexLabels.left_hand[1]][0][0]

In [ ]:
data[:, IndexLabels.right_hand[0]: IndexLabels.right_hand[1]]

In [ ]:
x_pts_rh = x_pts_lh

In [ ]:
lips = set()
for elem in mp.solutions.face_mesh_connections.FACEMESH_LIPS:
    lips.add(elem[0])
    lips.add(elem[1])

In [ ]:
plt.imshow(image)

Albumentations EDA

In [ ]:
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import albumentations as A
import matplotlib.pyplot as plt


import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

TOTAL_PTS = 543

In [ ]:
all_frames = np.load("./data/generated/np_landmarks/100015657.npy")
all_frames = np.resize(all_frames, (all_frames.shape[0], TOTAL_PTS, 3))
all_frames = np.where(np.isnan(all_frames), 0, all_frames)

In [ ]:
def apply_transform(transform, frame_keypoints):
    xy = [(np.max((pt[0], 0)), np.max((pt[1], 0))) for pt in frame_keypoints]

    augmented = transform(
        image=np.zeros((256, 256)),
        keypoints=xy,
        keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)
    )
    
    augmented_keypoints = np.array([[kp[0], kp[1]] for kp in augmented['keypoints']])
    
    # Appending z axis data to the augmented dataset
    augmented_keypoints = np.stack(
        [augmented_keypoints[:, 0], augmented_keypoints[:, 1], frame_keypoints[:, 2]],
        axis=-1
    )
    
    return augmented, augmented_keypoints

In [ ]:
transform = A.Compose([
#         A.Rotate(limit=10, p=1),
#         A.RandomCrop(width=128, height=128),
        A.RandomScale(p=1, scale_limit=(-1, 0.3)),
#         A.ShiftScaleRotate(p=1.0, shift_limit=10, border_mode=cv2.BORDER_CONSTANT, value=0),
#         A.RandomShear(p=1, shear_limit=0.2),
#         A.Affine(
#             scale=(-1, 0.5), keep_ratio=True, 
#             translate_percent=0.5,
#             rotate=10,
#             p=1.0,
#             shear=45,
#             mode=cv2.BORDER_CONSTANT
#         )
    ],
    keypoint_params=A.KeypointParams(format='xy')
)

frame_keypoints = all_frames[0]
xy = [(np.max((pt[0], 0)), np.max((pt[1], 0))) for pt in frame_keypoints]

augmented = transform(
    image=np.zeros((256, 256)),
    keypoints=xy,
    keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)
)

In [ ]:
all_frames.shape

In [ ]:
augmented["keypoints"]

In [ ]:
augmented_keypoints = np.array([[kp[0], kp[1]] for kp in augmented['keypoints']])

# Appending z axis data to the augmented dataset
augmented_keypoints = np.stack(
    [augmented_keypoints[:, 0], augmented_keypoints[:, 1], frame_keypoints[:, 2]],
    axis=-1
)

In [ ]:
transform = A.ReplayCompose([
        # A.HorizontalFlip(p=1),
        A.Rotate(limit=10, p=1),
#         A.RandomScale(p=1, scale_limit=(-1, 0.3)),
        # A.RandomTranslate(p=0.2, translate_limit=(-0.2, 0.2)),
        # A.RandomShear(p=1, shear_limit=0.2),
        # A.RandomRotate90(p=0.2),
    ],
    keypoint_params=A.KeypointParams(format='xy')
)

In [ ]:
np.arange(0, 10)

In [ ]:
aug_frames = []

augmented, augmented_keypoints = apply_transform(transform, all_frames[0])
replay_params = augmented["replay"]
aug_frames.append(augmented_keypoints)

for frame_keypoints in tqdm(all_frames[1:]):
    xy = [(np.max((pt[0], 0)), np.max((pt[1], 0))) for pt in frame_keypoints]
    
    augmented = A.ReplayCompose.replay(
        replay_params,
        image=np.zeros((256, 256)),
        keypoints=xy,
        keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)
    )

    augmented_keypoints = np.array([[kp[0], kp[1]] for kp in augmented['keypoints']])

    # Appending z axis data to the augmented dataset
    augmented_keypoints = np.stack(
        [augmented_keypoints[:, 0], augmented_keypoints[:, 1], frame_keypoints[:, 2]],
        axis=-1
    )

    # keypoints_aug[:, :2] /= [256, 256]

    aug_frames.append(augmented_keypoints)

aug_frames = np.array(aug_frames)

In [ ]:
augmented.keys()

In [ ]:
xy[1], augmented_keypoints[1]

In [ ]:
all_frames.shape, aug_frames.shape

In [ ]:
from src.visualize import get_pts_to_img

In [ ]:
img = get_pts_to_img(augmented_keypoints)

In [ ]:
augmented_keypoints[:, :2] /= [256, 256]

In [ ]:
all_frames[0]

In [ ]:
augmented_keypoints

In [ ]:
plt.imshow(img)

In [ ]:
aug_frames = np.array(augmented["keypoints"])

In [ ]:
i = 0
for r, a in zip(all_frames, aug_frames):
    img = get_pts_to_img(r)
    aug_img = get_pts_to_img(a)
    plt.figure()
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[1].imshow(aug_img)
    
    i += 1
    
    if i>=1:
        break

Horizontal flip EDA

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

TOTAL_PTS = 543

In [ ]:
all_frames = np.load("./data/generated/np_landmarks/100015657.npy")
all_frames = np.resize(all_frames, (all_frames.shape[0], TOTAL_PTS, 3))

In [ ]:
from src.visualize import get_pts_to_img
from src.augmentations import hflip

In [ ]:
aug_frames = hflip(all_frames)

In [ ]:
from src.augmentations import KeypointsNonFlatIndex

In [ ]:
idx, iax = 4, 0

In [ ]:
data = all_frames.copy()

data[:, :, 0] = 1 - data[:, :, 0]
aug_frames = data

In [ ]:
all_frames[idx][KeypointsNonFlatIndex.right_hand, iax], all_frames[idx][KeypointsNonFlatIndex.left_hand, iax]

In [ ]:
all_frames[idx][KeypointsNonFlatIndex.pose_right_hand, iax],\
all_frames[idx][KeypointsNonFlatIndex.pose_left_hand, iax]

In [ ]:
aug_frames[idx][KeypointsNonFlatIndex.pose_right_hand, iax],\
aug_frames[idx][KeypointsNonFlatIndex.pose_left_hand, iax]

In [ ]:
aug_frames[idx][KeypointsNonFlatIndex.right_hand, iax], aug_frames[idx][KeypointsNonFlatIndex.left_hand, iax]

In [ ]:
i = 0
for r, a in zip(all_frames, aug_frames):
    img = get_pts_to_img(r)
    aug_img = get_pts_to_img(a)

    plt.figure()
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[1].imshow(aug_img)
    
    i += 1
    
    if i>=10:
        break

Limited keypoints

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.visualize import get_pts_to_img
from src.augmentations import hflip
from src.keypoints import KeypointsNonFlatIndex

TOTAL_PTS = 543

In [ ]:
all_frames = np.load("./data/generated/np_landmarks/100015657.npy")
all_frames = np.where(np.isnan(all_frames), 0, all_frames)
all_frames = np.resize(all_frames, (all_frames.shape[0], TOTAL_PTS, 3))

In [ ]:
all_frames.shape

In [ ]:
all_frames = all_frames[
    :, list(KeypointsNonFlatIndex.lips) + list(KeypointsNonFlatIndex.eyes) + list(KeypointsNonFlatIndex.pose) + 
    list(KeypointsNonFlatIndex.left_hand) + list(KeypointsNonFlatIndex.right_hand), :
]

print(all_frames.shape)

In [ ]:
img = get_pts_to_img(all_frames[43])
plt.imshow(img)

Augmentations

In [ ]:
import albumentations as A

In [ ]:
class_labels = []
class_labels += ["face"]*len(KeypointsNonFlatIndex.face)
class_labels += ["left_hand"]*len(KeypointsNonFlatIndex.left_hand)
class_labels += ["pose"]*len(KeypointsNonFlatIndex.pose)
class_labels += ["right_hand"]*len(KeypointsNonFlatIndex.right_hand)

In [ ]:
targets_for_alb=all_frames.shape[0]
targets_for_alb=[f'keypoints{i}' for i in range(1, targets_for_alb)]
targets_for_alb=dict.fromkeys(targets_for_alb, 'keypoints')

In [ ]:
transform = A.Compose([
        A.Rotate(limit=20, p=1),
        A.RandomScale(scale_limit=(-1, 0.3), p=1),
        A.Affine(translate_percent=(0.01, 0.1), p=1)
    ],
    keypoint_params=A.KeypointParams(
        format='xy',
        remove_invisible=False
    ), 
    additional_targets=targets_for_alb
)

In [ ]:
# Converting negative value points to 0
all_frames_tfmd = np.maximum(all_frames, 0)
# Multiplying by 512 for albumentations to work properly
all_frames_tfmd = all_frames * 512

xy = all_frames_tfmd[:, :, :2]
print(xy.shape)

In [ ]:
keypointsxy = xy.shape[0]
keypointsxy = [f'keypoints{i}' for i in range(1, keypointsxy)]
keypointsxy = dict.fromkeys(keypointsxy, None)

for k, v in zip(keypointsxy.keys(), xy[1:, :, :]):
    keypointsxy[k] = v

In [ ]:
%%time
augmented = transform(
    image=np.zeros((512, 512)),
    keypoints=xy[0],
    **keypointsxy
)

In [ ]:
all_augmented_keypoints = []
i = 0
for k, v in augmented.items():
    if "keypoints" in k:
        temp = np.array(v)
        temp = np.stack(
            [temp[:, 0], temp[:, 1], all_frames_tfmd[i, :, 2]],
            axis=-1
        )
        all_augmented_keypoints.append(temp)
        
all_augmented_keypoints = np.array(all_augmented_keypoints)
all_augmented_keypoints /= 512

In [ ]:
# augmented = transform(image=np.zeros((512, 512)), keypoints=xy)
augmented_keypoints = np.array(augmented["keypoints"])
augmented_keypoints = np.stack(
    [augmented_keypoints[:, 0], augmented_keypoints[:, 1], all_frames_tfmd[0, :, 2]],
    axis=-1
)

# Scaling back for plotting
augmented_keypoints /= 512
print(augmented_keypoints.shape)

In [ ]:
i = 0

for img1, img2 in zip(all_frames, all_augmented_keypoints):
    img = get_pts_to_img(img1)
    aug_img = get_pts_to_img(img2)

    plt.figure()
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[1].imshow(aug_img)
    
    i += 1
    
    if i>=10:
        break

Numpy augmentations

In [ ]:
# Define augmentations
image_width, image_height = 512, 512

# Rotation degree
angle = np.radians(90)
cos = np.cos(angle)
sin = np.sin(angle)
rotation_matrix = np.array([[cos, -sin], [sin, cos]])

# Scaling factor
scale_factor = 0.5
scaling_matrix = np.eye(2) * scale_factor

# Center the keypoints
center_point = np.array([image_width/2, image_height/2])
keypoints_centered = all_frames_tfmd[:, :, :2] - center_point

In [ ]:
# augmented_keypoints = np.dot(keypoints_centered, rotation_matrix.T)
augmented_keypoints = np.dot(keypoints_centered, scaling_matrix.T)


# Add center point coordinates back to all keypoints
augmented_keypoints = augmented_keypoints + center_point

In [ ]:
augmented_keypoints = np.stack([
    augmented_keypoints[:, :, 0],
    augmented_keypoints[:, :, 1],
    all_frames_tfmd[:, :, 2]
], axis = -1)

augmented_keypoints /= 512

In [ ]:
i = 0

for img1, img2 in zip(all_frames, augmented_keypoints):
    img = get_pts_to_img(img1)
    aug_img = get_pts_to_img(img2)

    plt.figure()
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[1].imshow(aug_img)
    
    i += 1
    
    if i>=10:
        break

Flipping hand pose

[left hand x 3, right hand x3]
x-axis -> [20, 21, 22, 0, 0, 0]
img width flip (50)

x-axis -> [30, 29, 28, 50, 50, 50]

final -> [50, 50, 50, 30, 29, 28]

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

TOTAL_PTS = 543

from src.visualize import get_pts_to_img
from src.augmentations import hflip
from src.keypoints import KeypointsNonFlatIndex

In [ ]:
all_frames = np.load("./data/generated/np_landmarks/100015657.npy")
all_frames = np.where(np.isnan(all_frames), 0, all_frames)
all_frames = np.resize(all_frames, (all_frames.shape[0], TOTAL_PTS, 3))

all_frames.shape

In [ ]:
tmp = data[
    :, list(KeypointsNonFlatIndex.pose_right_hand), :
]

In [ ]:
tmp = all_frames[
    :, list(KeypointsNonFlatIndex.pose_left_hand)\
    + list(KeypointsNonFlatIndex.pose_right_hand), :
]

In [ ]:
img = get_pts_to_img(tmp[0])
plt.imshow(img)

In [ ]:
data = all_frames.copy()

In [ ]:
rh = np.argwhere(data[0, :, 0] > 0.5)
lh = np.argwhere(data[0, :, 0] <= 0.5)

In [ ]:
tmpx, tmpy, tmpz = data[0, rh, 0], data[0, rh, 1], data[0, rh, 2]

data[0, rh, 0] = 1 - data[0, lh, 0]
data[0, lh, 0] = 1 - tmpx

data[0, rh, 1] = data[0, lh, 1]
data[0, lh, 1] = tmpy

data[0, rh, 2] = data[0, lh, 2]
data[0, lh, 2] = tmpz

In [ ]:
lh.shape

In [ ]:
rh.shape

In [ ]:


x_pts_rh, y_pts_rh, z_pts_rh = data[:, KeypointsNonFlatIndex.right_hand, 0], \
    data[:, KeypointsNonFlatIndex.right_hand, 1], \
    data[:, KeypointsNonFlatIndex.right_hand, 2]

x_pts_lh, y_pts_lh, z_pts_lh = data[:, KeypointsNonFlatIndex.left_hand, 0], \
    data[:, KeypointsNonFlatIndex.left_hand, 1], \
    data[:, KeypointsNonFlatIndex.left_hand, 2]

In [ ]:
data[:, KeypointsNonFlatIndex.right_hand, 0] = 1 - x_pts_lh
data[:, KeypointsNonFlatIndex.left_hand, 0] = 1 - x_pts_rh

data[:, KeypointsNonFlatIndex.right_hand, 1] = y_pts_lh
data[:, KeypointsNonFlatIndex.left_hand, 1] = y_pts_rh

data[:, KeypointsNonFlatIndex.right_hand, 2] = z_pts_lh
data[:, KeypointsNonFlatIndex.left_hand, 2] = z_pts_rh

In [ ]:
data[0, KeypointsNonFlatIndex.right_hand, 0]

In [ ]:
data[0, KeypointsNonFlatIndex.left_hand, 0]

Model architecture

In [ ]:
import tensorflow as tf

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.add = tf.keras.layers.Add()
        self.normalize = tf.keras.layers.Normalization()

class GlobalSelfAttention(BaseAttention):
    def __call__(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x
        )

#         x = self.add([x, attn_output])
        x = self.normalize(attn_output)

        return x

In [ ]:
input_shape = (32, 294)
model_input = tf.keras.Input(shape=input_shape)

x = GlobalSelfAttention(num_heads=4, key_dim=128, output_shape=128, dropout=0.25)(model_input)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)

output = tf.keras.layers.Dense(250, activation="softmax")(x)

model = tf.keras.Model(model_input, output)

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

In [ ]:
model.summary()

In [ ]:
num_heads = 2
input_n = 32
input_dim = 1629
key_dim = 128
output_shape = 128

total_params = 0
# query, key, value projection
total_params += 3 * (input_dim*key_dim + key_dim)
print(total_params)
# total heads
total_params *= num_heads
print(total_params)
# downsclaing
total_params += key_dim*num_heads*output_shape + output_shape
print(total_params)

Verfifying data generation

In [ ]:
from src.data_generator import GISLRSequence
import src.utils as utils

In [ ]:
save_dataset_path = "./data/generated/np_landmarks/"
class_path = "./data/raw/train.csv"
class_to_label_path = "./data/raw/sign_to_prediction_index_map.json"
val_participants = [55372, 61333, 62590]
test_participants = [30680, 37779, 2044]


In [ ]:
df = utils.get_train_dataset(
    save_dataset_path=save_dataset_path,
    class_path=class_path,
    class_to_label_path=class_to_label_path
)

df = utils.get_example_set(
    df, test_participants=test_participants, val_participants=val_participants
)

In [ ]:
train_dataset = GISLRSequence(
    df[df.example_set == "train"],
    x_col="save_dataset_path",
    y_col="y_label",
    sample_size=32,
    batch_size=16
)

In [ ]:
for x, y in train_dataset:
    break

In [ ]:
from src.raw_data_manipulation.visualize import get_pts_to_img
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
for elem in x[0]:
    plt.figure()
    img = get_pts_to_img(np.resize(elem, (543, 3)))
    plt.imshow(img)